In [10]:
# -*- coding: utf-8 -*-

'''

每个被试能够 recall 或者 recognition的单词数量是不确定的，

这里每个condition都是对应的一个epoch trial。所以噪音很大。
'''

__author__ = ' liangyaowen '


'''
这里是想做单试次的分析吗？

这样子虽然能增加统计的基数。。相比于ERP。。

'''


'\n这里是想做单试次的分析吗？\n\n这样子虽然能增加统计的基数。。相比于ERP。。\n\n'

In [11]:
import numpy as np
import os.path as op
from pandas import read_csv
import mne
from mne.io import read_raw_fif
from mne.datasets import visual_92_categories
from neurora.nps_cal import nps
from neurora.rdm_cal import eegRDM
from neurora.rdm_corr import rdm_correlation_spearman
from neurora.corr_cal_by_rdm import rdms_corr
from neurora.rsa_plot import plot_rdm, plot_corrs_by_time, plot_nps_hotmap, plot_corrs_hotmap

In [15]:
"""
数据应该是从epochs里读，每个epoch的标签不一样才对
简化：
先读一个被试的数据看看
每个epoch对应一个condition（关键的是数量不对等问题-不同条件epoch数量不等，要不要随机抽取？假设每个抽取12个）

"""
T1_sub_ids = [
            'prex009'  ]
#去掉了 5和9被试
T2M_sub_ids = [
            'prex009M' ]

T2S_sub_ids = [
            'prex009S' ]
data_path = 'E:\Python LYW\RSA\spaced_learning\pre10\\analysis\data\\6epoch_clean\\'

list_epochs_T1S = list()
list_epochs_T2S = list()
list_epochs_T2M = list()

for sub_id in T1_sub_ids:
    fname = data_path + sub_id + '-epo.fif'
    epochs = mne.read_epochs(fname)
    list_epochs_T1S.append(epochs)

for sub_id in T2S_sub_ids:
    fname = data_path + sub_id + '-epo.fif'
    epochs = mne.read_epochs(fname)
    list_epochs_T2S.append(epochs)

for sub_id in T2M_sub_ids:
    fname = data_path + sub_id + '-epo.fif'
    epochs = mne.read_epochs(fname)
    list_epochs_T2M.append(epochs)

Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\6epoch_clean\prex009-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
39 matching events found
No baseline correction applied
0 projection items activated
Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\6epoch_clean\prex009S-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
44 matching events found
No baseline correction applied
0 projection items activated
Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\6epoch_clean\prex009M-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
44 matching events found
No baseline correction applied
0 pr

In [16]:
# 制作出完整的epochs
import random
epochs_3types = list_epochs_T1S[0]
for i in random.sample(range(0,len(list_epochs_T2S[0].events)),len(list_epochs_T1S[0].events)):
    epochs_3types = mne.concatenate_epochs([epochs_3types,list_epochs_T2S[0][i]])
for j in random.sample(range(0,len(list_epochs_T2M[0].events)),len(list_epochs_T1S[0].events)):
    epochs_3types = mne.concatenate_epochs([epochs_3types,list_epochs_T2M[0][j]])


Not setting metadata
Not setting metadata
40 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
41 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
42 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
43 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
44 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
45 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
46 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Not setting metadata
Not setting metadata
47 matching events found
Applying baseline corre

In [17]:
# the shape of MEG data: megdata is [3, 92, 306, 1101]
# n_subs = 3, n_conditions = 92, n_channels = 306, n_timepoints = 1101 (-100ms to 1000ms)
subindex = 0
#注意第一个维度是subs
eegdata = np.zeros([len(T1_sub_ids),len(epochs_3types), 63, 501], dtype=np.float32)
#第一个维度是condition
subdata = np.zeros([len(epochs_3types), 63, 501], dtype=np.float32)
for i in range(len(epochs_3types)):
    data = epochs_3types[i].average().data
    subdata[i] = data
eegdata[subindex] = subdata

In [18]:
# shape of megdata: [n_subs, n_cons, n_chls, n_ts] -> [n_cons, n_subs, n_chls, n_ts]
eegdata = np.transpose(eegdata, (1, 0, 2, 3))

# shape of megdata: [n_cons, n_subs, n_chls, n_ts] -> [n_cons, n_subs, n_trials, n_chls, n_ts]
# here data is averaged, so set n_trials = 1
eegdata = np.reshape(eegdata, [len(epochs_3types), len(T1_sub_ids), 1, 63, 501])

In [ ]:
# Section 4: Calculating single RDM and Plotting
# Calculate the RDM based on the data during 190ms-210ms
rdm = eegRDM(eegdata[:, :, :, :, 290:310])


' a function for plotting the RDM '

import matplotlib.pyplot as plt

def plot_rdm(rdm, percentile=False, rescale=False, lim=[0, 1], conditions=None, con_fontsize=12, cmap=None):

    """
    Plot the RDM

    Parameters
    ----------
    rdm : array or list [n_cons, n_cons]
        A representational dissimilarity matrix.
    percentile : bool True or False. Default is False.
        Rescale the values in RDM or not by displaying the percentile.
    rescale : bool True or False. Default is False.
        Rescale the values in RDM or not.
        Here, the maximum-minimum method is used to rescale the values except for the
        values on the diagnal.
    lim : array or list [min, max]. Default is [0, 1].
        The corrs view lims.
    conditions : string-array or string-list. Default is None.
        The labels of the conditions for plotting.
        conditions should contain n_cons strings, If conditions=None, the labels of conditions will be invisible.
    con_fontsize : int or float. Default is 12.
        The fontsize of the labels of the conditions for plotting.
    cmap : matplotlib colormap. Default is None.
        The colormap for RDM.
        If cmap=None, the ccolormap will be 'jet'.
    """

    if len(np.shape(rdm)) != 2 or np.shape(rdm)[0] != np.shape(rdm)[1]:

        return "Invalid input!"

    # get the number of conditions
    cons = rdm.shape[0]

    # if cons=2, the RDM cannot be plotted.
    if cons == 2:
        print("The shape of RDM cannot be 2*2. Here NeuroRA cannot plot this RDM.")

        return None

    # determine if it's a square
    a, b = np.shape(rdm)
    if a != b:
        return None

    if percentile == True:

        v = np.zeros([cons * cons, 2], dtype=np.float)
        for i in range(cons):
            for j in range(cons):
                v[i * cons + j, 0] = rdm[i, j]

        index = np.argsort(v[:, 0])
        m = 0
        for i in range(cons * cons):
            if i > 0:
                if v[index[i], 0] > v[index[i - 1], 0]:
                    m = m + 1
                v[index[i], 1] = m

        v[:, 0] = v[:, 1] * 100 / m

        for i in range(cons):
            for j in range(cons):
                rdm[i, j] = v[i * cons + j, 0]

        if cmap == None:
            plt.imshow(rdm, extent=(0, 1, 0, 1), cmap=plt.cm.jet, clim=(0, 100))
        else:
            plt.imshow(rdm, extent=(0, 1, 0, 1), cmap=cmap, clim=(0, 100))

    # rescale the RDM
    elif rescale == True:

        # flatten the RDM
        vrdm = np.reshape(rdm, [cons * cons])
        # array -> set -> list
        svrdm = set(vrdm)
        lvrdm = list(svrdm)
        lvrdm.sort()

        # get max & min
        maxvalue = lvrdm[-1]
        minvalue = lvrdm[1]

        # rescale
        if maxvalue != minvalue:

            for i in range(cons):
                for j in range(cons):

                    # not on the diagnal
                    if i != j:
                        rdm[i, j] = float((rdm[i, j] - minvalue) / (maxvalue - minvalue))

        # plot the RDM
        min = lim[0]
        max = lim[1]
        if cmap == None:
            plt.imshow(rdm, extent=(0, 1, 0, 1), cmap=plt.cm.jet, clim=(min, max))
        else:
            plt.imshow(rdm, extent=(0, 1, 0, 1), cmap=cmap, clim=(min, max))

    else:

        # plot the RDM
        min = lim[0]
        max = lim[1]
        if cmap == None:
            plt.imshow(rdm, extent=(0, 1, 0, 1), cmap=plt.cm.jet, clim=(min, max))
        else:
            plt.imshow(rdm, extent=(0, 1, 0, 1), cmap=cmap, clim=(min, max))

    # plt.axis("off")
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=16)
    font = {'size': 18}

    if percentile == True:
        cb.set_label("Dissimilarity (percentile)", fontdict=font)
    elif rescale == True:
        cb.set_label("Dissimilarity (Rescaling)", fontdict=font)
    else:
        cb.set_label("Dissimilarity", fontdict=font)

    if conditions != None:
        print("1")
        step = float(1 / cons)
        x = np.arange(0.5 * step, 1 + 0.5 * step, step)
        y = np.arange(1 - 0.5 * step, -0.5 * step, -step)
        plt.xticks(x, conditions, fontsize=con_fontsize, rotation=30, ha="right")
        plt.yticks(y, conditions, fontsize=con_fontsize)
    else:
        plt.axis("off")

    plt.show()

    return 0

# Plot this RDM
%matplotlib qt
plot_rdm(rdm, percentile=True)

In [ ]:
#Section 5: Calculating RDMs and Plotting
# Calculate the RDMs by a 10ms time-window
# (raw sampling requency is 1000Hz, so here time_win=10ms/(1s/1000Hz)/1000=10)
rdms = eegRDM(eegdata, time_opt=1, time_win=200, time_step=10)

# Plot the RDM of -100ms, 0ms, 50ms, 100ms, 150ms, 200ms
times = [0, 10, 20, 30, 40]
%matplotlib inline

for t in times:
    plot_rdm(rdms[t], percentile=True)

In [ ]:
#Section 6: Calculating the Similarity between two RDMs
# RDM of 200ms
rdm_sample1 = rdms[30]
# RDM of 800ms
rdm_sample2 = rdms[20]

# calculate the correlation coefficient between these two RDMs
corr = rdm_correlation_spearman(rdm_sample1, rdm_sample2)
print(corr)



In [ ]:
# 先考虑造小的，再考虑拼接
shape = (len(list_epochs_T1S[0]),len(list_epochs_T1S[0]))
sigle_matrix1 = np.zeros(shape, dtype=np.float32)
sigle_matrix2 = np.zeros(shape, dtype=np.float32)
sigle_matrix3 = np.zeros(shape, dtype=np.float32)
sigle_matrix4 = np.zeros(shape, dtype=np.float32)
sigle_matrix1[:,:] = 1-1
sigle_matrix2[:,:] = 1-0.6
sigle_matrix3[:,:] = 1-0.3
sigle_matrix4[:,:] = 1-0.5

In [ ]:
# 构造出 自己要的model矩阵
sum_matrix_row1 = np.concatenate((sigle_matrix1,sigle_matrix2,sigle_matrix3))
sum_matrix_row2 = np.concatenate((sigle_matrix2,sigle_matrix1,sigle_matrix4))
sum_matrix_row3 = np.concatenate((sigle_matrix3,sigle_matrix4,sigle_matrix1))
sum_matrix = np.concatenate((sum_matrix_row1,sum_matrix_row2,sum_matrix_row3),axis=1)
model_matrix = sum_matrix

In [ ]:
#滑窗，让每个时间窗都和model_matrix进行相关。
# Calculate the representational similarity between 200ms and all the time points
corrs1 = rdms_corr(model_matrix, rdms)
# Plot the corrs1
# 加了一个维度？
corrs1 = np.reshape(corrs1, [1, 31, 2])
plot_corrs_by_time(corrs1, time_unit=[-0.1, 0.01])